In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [2]:
walmart_base_dir = r'C:\Users\s346557\Documents\LocalData\walmart'

In [3]:
walmart_pns = pd.read_excel(os.path.join(walmart_base_dir, 'walmart_premises.xlsx'), dtype=str)

In [4]:
walmart_pns

,prem_nb,co_cd_ownr
0,010443928,01
1,015077836,01
2,016571053,01
3,018704260,01
4,027484939,02
...,...,...
446,977342979,97
447,978039827,97
448,978372565,97
449,979899029,97


In [5]:
walmart_pns['prem_nb'].nunique()

451

In [6]:
date_0 = '2022-01-01'
date_1 = '2023-04-30'
# date_1 = '2022-01-07'

In [7]:
save_end_events=True

In [8]:
#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------
# DFs will be saved in save_dir_base
# Collection of end events files will be saved in os.path.join(save_dir_base, 'EndEvents')
save_dir_base = os.path.join(
    walmart_base_dir, 
    f"{date_0.replace('-','')}_{date_1.replace('-','')}"
)
#-------------------------
end_events_save_args = dict(
    save_to_file=save_end_events, 
    save_dir = os.path.join(save_dir_base, 'EndEvents'), 
    save_name=r'end_events.csv', 
    index=True
)
#-------------------------
print(f"save_dir_base = {save_dir_base}")
print('end_events_save_args')
for k,v in end_events_save_args.items():
    print(f"\t{k} : {v}")
#-------------------------
if save_end_events:
    if not os.path.exists(end_events_save_args['save_dir']):
        os.makedirs(end_events_save_args['save_dir'])

save_dir_base = C:\Users\s346557\Documents\LocalData\walmart\20220101_20230430
end_events_save_args
	save_to_file : True
	save_dir : C:\Users\s346557\Documents\LocalData\walmart\20220101_20230430\EndEvents
	save_name : end_events.csv
	index : True


In [9]:
#-------------------------
usg_split_to_CTEs=True
df_construct_type=DFConstructType.kRunSqlQuery
contstruct_df_args_end_events=None
addtnl_groupby_cols=None

cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
batch_size=10
verbose=True
n_update=1
#-------------------------
end_events_sql_function=AMIEndEvents_SQL.build_sql_end_events
end_events_sql_function_kwargs = dict(
    cols_of_interest=cols_of_interest_end_dev_event, 
    premise_nbs=walmart_pns['prem_nb'].unique().tolist(), 
    date_range=[date_0, date_1], 
#     field_to_split='premise_nbs', 
    field_to_split=None, 
#     save_and_dump=True, 
    save_and_dump=False, 
    sort_coll_to_split=True,
    batch_size=batch_size, 
    verbose=verbose,
    n_update=n_update
)

In [10]:
start=time.time()
end_events = AMIEndEvents(
    df_construct_type=df_construct_type, 
    contstruct_df_args = contstruct_df_args_end_events, 
    build_sql_function=end_events_sql_function, 
    build_sql_function_kwargs=end_events_sql_function_kwargs, 
    init_df_in_constructor=True, 
    save_args=end_events_save_args
)
end_events_build_time = time.time()-start

# ==================================================

In [12]:
ede_df = end_events.df.copy()
#-----
ede_df['reason_full'] = ede_df['reason']
ede_df = AMIEndEvents.reduce_end_event_reasons_in_df(df=ede_df)
#-----
ede_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
    df=ede_df, 
    time_col='valuesinterval', 
    placement_col='valuesinterval_local', 
    run_quick=True, 
    n_strip=6, 
    inplace=False
)

In [13]:
freq='D'
group_cols=['serialnumber', 'aep_premise_nb']
group_freq=pd.Grouper(freq=freq, key='valuesinterval_local')
#-------------------------
rcpx_df = AMIEndEvents.get_reason_counts_per_group(
    end_events_df=ede_df, 
    group_cols=group_cols, 
    group_freq=group_freq, 
    inclue_zero_counts=True, 
    include_nSNs=False, 
    include_SNs=False, 
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False) 
)

In [17]:
natsorted(rcpx_df.columns.tolist())

['Demand Reset',
 'Detected end of voltage sag',
 'Detected end of voltage swell',
 'Device exceeded the max allowable trap threshold',
 'Diag1 Condition cleared',
 'Diag1: Polarity, Cross Phase, Reverse Energy Flow: Angle out of tolerance.',
 'Diag6 Condition cleared',
 'Diag6: Under Voltage, Element A',
 'Diag7 Condition cleared',
 'Diag7: Over Voltage, Element A ',
 'Error occurred when attempting to synch meter time with NIC time for device',
 'KV2c meter event Over Voltage (Diagnostic flags = Phase A Voltage)',
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow',
 'KV2c meter event Under Voltage  (Diagnostic flags = Phase A Voltage)',
 'Last Gasp - NIC power lost for device, Fail Reason: [0x40] LG_DIRECT_NOTIFICATION',
 'Last Gasp - NIC power lost for device, Fail Reason: [0x41] LG_DIRECT_NOTIFICATION',
 'Last Gasp - NIC power lost for device, Fail Reason: [0x48] LG_ZERO_X_DETECTOR ,LG_DIRECT_NOTIFICATION',
 'Last Gasp - NIC power lost for device, Fail Reason: [0x49] LG

In [16]:
ede_df[ede_df['reason'].str.contains('Under Voltage')]

,issuertracking_id,serialnumber,enddeviceeventtypeid,valuesinterval,aep_premise_nb,reason,event_type,aep_opco,aep_event_dt,reason_full,valuesinterval_local
5,1-3964478848,561485363,3.26.38.150,2022-11-16T06:35:15-06:00,957171813,Under Voltage (CA000400) Phase C Voltage out...,3,pso,2022-11-16,Under Voltage (CA000400) for meter 00:13:50:03...,2022-11-16 06:35:15
6,1-3964478851,561485363,3.26.38.37,2022-11-16T06:42:20-06:00,957171813,Under Voltage (CA000400) cleared,3,pso,2022-11-16,Under Voltage cleared (CA000400) for meter 00:...,2022-11-16 06:42:20
7,1-3965551852,561485363,3.26.38.150,2022-11-16T18:51:59-06:00,957171813,Under Voltage (CA000400) Phase C Voltage out...,3,pso,2022-11-16,Under Voltage (CA000400) for meter 00:13:50:03...,2022-11-16 18:51:59
8,1-3964478847,561485363,3.26.38.37,2022-11-16T06:20:30-06:00,957171813,Under Voltage (CA000400) cleared,3,pso,2022-11-16,Under Voltage cleared (CA000400) for meter 00:...,2022-11-16 06:20:30
10,1-3965551850,561485363,3.26.38.150,2022-11-16T17:31:49-06:00,957171813,Under Voltage (CA000400) Phase C Voltage out...,3,pso,2022-11-16,Under Voltage (CA000400) for meter 00:13:50:03...,2022-11-16 17:31:49
...,...,...,...,...,...,...,...,...,...,...,...
484468,5-4496367728,590083819,3.26.38.150,2022-12-24T00:15:27.000-05:00,026512178,Under Voltage (CA000400) Phase B Voltage out...,3,ap,2022-12-24,Under Voltage (CA000400) for meter 00:13:50:05...,2022-12-24 00:15:27
484469,5-4496367732,590083819,3.26.38.37,2022-12-24T00:18:07.000-05:00,026512178,Under Voltage (CA000400) cleared,3,ap,2022-12-24,Under Voltage cleared (CA000400) for meter 00:...,2022-12-24 00:18:07
484470,5-4496367782,590083819,3.26.38.150,2022-12-24T01:35:57.000-05:00,026512178,Under Voltage (CA000400) Phase B Voltage out...,3,ap,2022-12-24,Under Voltage (CA000400) for meter 00:13:50:05...,2022-12-24 01:35:57
484488,5-4502057388,590083819,3.26.38.150,2022-12-24T06:05:54.000-05:00,026512178,Under Voltage (CA000400) Phase B Voltage out...,3,ap,2022-12-24,Under Voltage (CA000400) for meter 00:13:50:05...,2022-12-24 06:05:54


In [18]:
save_dir_base

'C:\\Users\\s346557\\Documents\\LocalData\\walmart\\20220101_20230430'

In [20]:
ede_df.to_pickle(os.path.join(save_dir_base, 'end_events.pkl'))
rcpx_df.to_pickle(os.path.join(save_dir_base, 'reason_counts.pkl'))

In [22]:
ede_df['reason'].nunique()

64

In [23]:
ede_df['reason_full'].nunique()

281705

In [24]:
ede_df.shape[0]

484558

In [26]:
rcpx_df.shape

(14868, 64)

In [27]:
rcpx_df

reason                                            Demand Reset  \
serialnumber aep_premise_nb valuesinterval_local                 
143447052    940934458      2022-07-05                     0.0   
                            2022-07-12                     0.0   
                            2022-07-13                     0.0   
                            2022-07-14                     0.0   
                            2022-07-15                     0.0   
...                                                        ...   
998695408    968966281      2023-03-16                     0.0   
                            2023-04-02                     0.0   
                            2023-04-14                     1.0   
                            2023-04-18                     0.0   
                            2023-04-25                     0.0   

reason                                            Detected end of voltage sag  \
serialnumber aep_premise_nb valuesinterval_local                                
143447052    940934458      2022-07-05                                    0.0   
                            2022-07-12                                    0.0   
                            2022-07-13                                    0.0   
                            2022-07-14                                    0.0   
                            2022-07-15                                    0.0   
...                                                                       ...   
998695408    968966281      2023-03-16                                    0.0   
                            2023-04-02                                    0.0   
                            2023-04-14                                    0.0   
                            2023-04-18                                    0.0   
                            2023-04-25                                    0.0   

reason                                            Detected end of voltage swell  \
serialnumber aep_premise_nb valuesinterval_local                                  
143447052    940934458      2022-07-05                                      0.0   
                            2022-07-12                                      0.0   
                            2022-07-13                                      0.0   
                            2022-07-14                                      0.0   
                            2022-07-15                                      0.0   
...                                                                         ...   
998695408    968966281      2023-03-16                                      0.0   
                            2023-04-02                                      0.0   
                            2023-04-14                                      0.0   
                            2023-04-18                                      0.0   
                            2023-04-25                                      0.0   

reason                                            Device exceeded the max allowable trap threshold  \
serialnumber aep_premise_nb valuesinterval_local                                                     
143447052    940934458      2022-07-05                                                         0.0   
                            2022-07-12                                                         0.0   
                            2022-07-13                                                         0.0   
                            2022-07-14                                                         0.0   
                            2022-07-15                                                         0.0   
...                                                                                            ...   
998695408    968966281      2023-03-16                                                         0.0   
                            2023-04-02                                                   